# Animate the n-body simulation

Here, we will follow instructions to execute an $n$-body simulation from Hut and Makino [here](https://www.ids.ias.edu/~piet/act/comp/algorithms/starter).

This is a third-party software so we will download it, compile it (it is in `C++`), and use it to execute a simulation. We will then use `python` tools to animate the trajectory after it has been simulated. 

## Download, compile

In [ ]:
! wget https://www.ids.ias.edu/sites/ids.ias.edu/files/imported/act/comp/algorithms/starter/nbody_sh1.tar.gz
! mkdir hut_makino
! tar -zxvf nbody_sh1.tar.gz --directory hut_makino
! g++ hut_makino/nbody_sh1.C -o hut_makino/nbody_sh1
! chmod +x hut_makino/nbody_sh1

### Execute a Figure-8

In [ ]:
! ./hut_makino/nbody_sh1 -o 0.01 < figure8.in > figure8.out

### Execute a random configuration

In [ ]:
! python generate_random_start.py > random.in
! ./hut_makino/nbody_sh1 -o 0.01 < random.in > random.out

## Make the animation

In [ ]:
import matplotlib
from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='html5')

import matplotlib.pyplot as plt
import numpy as np
from read_orbit import read_orbit

In [ ]:
class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, filename="figure8.out"):
        self.stream = self.data_stream()
        self.nobj, self.snapshots = read_orbit(filename)        
        
        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots()
        # Then setup FuncAnimation.
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=10, frames=500,
                                           init_func=self.setup_plot, blit=False)

    def setup_plot(self):

        """Initial drawing of the scatter plot."""
        x, y = next(self.stream).T
        self.scat = self.ax.scatter(x, y, animated=True)
        self.ax.axis([-10, 10, -10, 10])

        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,
    

    def data_stream(self):
        
        i = 0
        x = [0.] * self.nobj
        y = [0.] * self.nobj
        while True:
            if i >= len(self.snapshots):
                i = 0
            snapshot = self.snapshots[i]
            for iparticle,particle in enumerate(snapshot):
                x[iparticle] = particle.pos[0]
                y[iparticle] = particle.pos[1]
            i += 1
            yield np.c_ [x,y]
            

    def update(self, i):
        """Update the scatter plot."""
        data = next(self.stream)

        # Set x and y data...
        self.scat.set_offsets(data)

        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,


## Animate!

In [ ]:
a = AnimatedScatter(filename="figure8.out")        
HTML(a.ani.to_html5_video())

In [ ]:
a = AnimatedScatter(filename="random.out")        
HTML(a.ani.to_html5_video())